## Homework

> Note: sometimes your answer doesn't match one of 
> the options exactly. That's fine. 
> Select the option that's closest to your solution.



### Dataset

For this homework, we'll use the Car Fuel Efficiency dataset. Download it from <a href='https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv'>here</a>.

You can do it with wget:
```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
```

The goal of this homework is to create a regression model for predicting the car fuel efficiency (column `'fuel_efficiency_mpg'`).



### Preparing the dataset 

Use only the following columns:

* `'engine_displacement'`,
* `'horsepower'`,
* `'vehicle_weight'`,
* `'model_year'`,
* `'fuel_efficiency_mpg'`



### EDA

* Look at the `fuel_efficiency_mpg` variable. Does it have a long tail? 



In [36]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px

In [7]:
datapath = Path()
datapath.mkdir(parents=True, exist_ok=True)

url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
file_name = url.split("/")[-1]
if not (datapath / file_name).is_file():
    print(f"Downloading {file_name}")
    !wget $url
else :
    print(f"{file_name} already exists")

car_fuel_efficiency.csv already exists


In [8]:
# read the data
df = pd.read_csv(datapath / file_name)
df.head()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9704 entries, 0 to 9703
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   engine_displacement  9704 non-null   int64  
 1   num_cylinders        9222 non-null   float64
 2   horsepower           8996 non-null   float64
 3   vehicle_weight       9704 non-null   float64
 4   acceleration         8774 non-null   float64
 5   model_year           9704 non-null   int64  
 6   origin               9704 non-null   object 
 7   fuel_type            9704 non-null   object 
 8   drivetrain           9704 non-null   object 
 9   num_doors            9202 non-null   float64
 10  fuel_efficiency_mpg  9704 non-null   float64
dtypes: float64(6), int64(2), object(3)
memory usage: 834.1+ KB


In [38]:
# check if fuel_efficiency_mpg has a long tail
px.histogram(df, x='fuel_efficiency_mpg', nbins=50,height=400,width=600,marginal='box')\
    .update_layout(xaxis_title='Fuel Efficiency (MPG)', yaxis_title='Frequency',template='simple_white', bargap=0.1,title_x=0.5)

The `fuel_efficiency_mpg` **has no long tail** with normal distribution.

### Question 1

There's one column with missing values. What is it?

* `'engine_displacement'`
* **`'horsepower'`**
* `'vehicle_weight'`
* `'model_year'`




In [11]:
df.isnull().sum()

engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64

In [ ]:
cols = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']
df[cols].isnull().sum()

engine_displacement      0
horsepower             708
vehicle_weight           0
model_year               0
dtype: int64

### Question 2

What's the median (50% percentile) for variable `'horsepower'`?

- 49
- 99
- **`149`**
- 199



In [13]:
df.horsepower.describe()

count    8996.000000
mean      149.657292
std        29.879555
min        37.000000
25%       130.000000
50%       149.000000
75%       170.000000
max       271.000000
Name: horsepower, dtype: float64

### Prepare and split the dataset

* Shuffle the dataset (the filtered one you created above), use seed `42`.
* Split your data in train/val/test sets, with 60%/20%/20% distribution.

Use the same code as in the lectures




In [67]:
np.random.seed(42)
n_val = int(len(df) * 0.2)
n_test = int(len(df) * 0.2)
n_train = len(df) - n_val - n_test

idx = np.arange(len(df))
np.random.shuffle(idx)
df_shuffled = df.iloc[idx]

df_train = df_shuffled.iloc[:n_train].copy().reset_index(drop=True)
df_val = df_shuffled.iloc[n_train:n_train+n_val].copy().reset_index(drop=True)
df_test = df_shuffled.iloc[n_train+n_val:].copy().reset_index(drop=True)    

In [68]:
n_val+n_test+n_train , len(df)

(9704, 9704)

In [69]:
# set target
y_train = df_train.fuel_efficiency_mpg.values
y_val = df_val.fuel_efficiency_mpg.values
y_test = df_test.fuel_efficiency_mpg.values

# drop the target from the dataframes
df_train = df_train.drop(columns=['fuel_efficiency_mpg'])
df_val = df_val.drop(columns=['fuel_efficiency_mpg'])
df_test = df_test.drop(columns=['fuel_efficiency_mpg']) 

### Question 3

* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using `round(score, 2)`
* Which option gives better RMSE?

Options:

- With 0
- **`With mean`**
- Both are equally good




In [102]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return round(np.sqrt(mse), 2)
    # return np.sqrt(mse)

In [71]:
# fill missing values with 0
base = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']

def prepare_X(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

X_train = prepare_X(df_train)
w_0, w = train_linear_regression(X_train, y_train)

y_pred = w_0 + X_train.dot(w)
print('train', rmse(y_train, y_pred))

X_val = prepare_X(df_val)
y_pred = w_0 + X_val.dot(w)
print('validation', rmse(y_val, y_pred))

train 0.52
validation 0.52


In [72]:
# fill missing values with mean
base = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']

def prepare_X(df):
    df_num = df[base]
    df_num = df_num.fillna(df_num.mean())
    X = df_num.values
    return X

X_train = prepare_X(df_train)
w_0, w = train_linear_regression(X_train, y_train)

y_pred = w_0 + X_train.dot(w)
print('train', rmse(y_train, y_pred))

X_val = prepare_X(df_val)
y_pred = w_0 + X_val.dot(w)
print('validation', rmse(y_val, y_pred))

train 0.46
validation 0.46


### Question 4

* Now let's train a regularized linear regression.
* For this question, fill the NAs with 0. 
* Try different values of `r` from this list: `[0, 0.01, 0.1, 1, 5, 10, 100]`.
* Use RMSE to evaluate the model on the validation dataset.
* Round the RMSE scores to 2 decimal digits.
* Which `r` gives the best RMSE?

If there are multiple options, select the smallest `r`.

Options:

- 0
- **`0.01`**
- 1
- 10
- 100




In [83]:
def train_linear_regression_reg(X, y, r=0.0):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

# fill missing values with 0
base = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']

def prepare_X(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

In [91]:
X_train = prepare_X(df_train)
X_val = prepare_X(df_val)

for r in [0, 0.01, 0.1, 1, 5, 10, 100]:
    w_0, w = train_linear_regression_reg(X_train, y_train, r=r)
    y_pred = w_0 + X_val.dot(w)
    print('%6s' %r, rmse(y_val, y_pred))

     0 0.517378263883855
  0.01 0.5171115525770756
   0.1 0.5187525130700945
     1 0.5222348802092502
     5 0.5228916092823479
    10 0.5229812979636552
   100 0.5230636233819905


In [121]:
import numpy as np
import pandas as pd

# Step 1: Split data (using the same seed as before)
df_train, y_train, df_val, y_val, df_test, y_test = prepare_df(df, seed=42)

# Step 2: Fill missing with 0
X_train = prepare_X(df_train)
X_val = prepare_X(df_val)

# Step 3: Define the regularized linear regression function
def train_linear_regression_reg(X, y, r=0.0):
    # Add bias term
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    
    # Regularization matrix
    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0])
    XTX_inv = np.linalg.inv(XTX)
    
    w_full = XTX_inv.dot(X.T).dot(y)
    w_0 = w_full[0]
    w = w_full[1:]
    return w_0, w


# Step 4: Try different r values
r_values = [0, 0.01, 0.1, 1, 5, 10, 100]
results = []

for r in r_values:
    w_0, w = train_linear_regression_reg(X_train, y_train, r=r)
    
    y_pred = w_0 + X_val.dot(w)
    score = rmse(y_val, y_pred)
    
    print(f"r={r:<6} -> RMSE: {score:.2f}")
    
    results.append((r, round(score, 2)))


# Step 5: Find the best r
df_results = pd.DataFrame(results, columns=['r', 'val_rmse'])
best = df_results.loc[df_results['val_rmse'].idxmin()]

print("\nValidation RMSE results:")
print(df_results)

print(f"\n✅ Best r = {best['r']} with RMSE = {best['val_rmse']}")


r=0      -> RMSE: 0.52
r=0.01   -> RMSE: 0.52
r=0.1    -> RMSE: 0.52
r=1      -> RMSE: 0.52
r=5      -> RMSE: 0.52
r=10     -> RMSE: 0.52
r=100    -> RMSE: 0.52

Validation RMSE results:
        r  val_rmse
0    0.00      0.52
1    0.01      0.52
2    0.10      0.52
3    1.00      0.52
4    5.00      0.52
5   10.00      0.52
6  100.00      0.52

✅ Best r = 0.0 with RMSE = 0.52


### Question 5 

* We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
* Try different seed values: `[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]`.
* For each seed, do the train/validation/test split with 60%/20%/20% distribution.
* Fill the missing values with 0 and train a model without regularization.
* For each seed, evaluate the model on the validation dataset and collect the RMSE scores. 
* What's the standard deviation of all the scores? To compute the standard deviation, use `np.std`.
* Round the result to 3 decimal digits (`round(std, 3)`)

What's the value of std?

- 0.001
- **`0.006`**
- 0.060
- 0.600

> Note: Standard deviation shows how different the values are.
> If it's low, then all values are approximately the same.
> If it's high, the values are different. 
> If standard deviation of scores is low, then our model is *stable*.




In [117]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return round(np.sqrt(mse), 3)
    # return np.sqrt(mse)

In [118]:
# prepare dataframes
def prepare_df(df, seed=42):
    np.random.seed(seed)
    n_val = int(len(df) * 0.2)
    n_test = int(len(df) * 0.2)
    n_train = len(df) - n_val - n_test

    idx = np.arange(len(df))
    np.random.shuffle(idx)
    df_shuffled = df.iloc[idx]

    df_train = df_shuffled.iloc[:n_train].copy().reset_index(drop=True)
    df_val = df_shuffled.iloc[n_train:n_train+n_val].copy().reset_index(drop=True)
    df_test = df_shuffled.iloc[n_train+n_val:].copy().reset_index(drop=True)

    y_train = df_train.fuel_efficiency_mpg.values
    y_val = df_val.fuel_efficiency_mpg.values
    y_test = df_test.fuel_efficiency_mpg.values

    del df_train['fuel_efficiency_mpg']
    del df_val['fuel_efficiency_mpg']
    del df_test['fuel_efficiency_mpg']

    return df_train, y_train, df_val, y_val, df_test, y_test


# features
base = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']

def prepare_X(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X


# store results
result_list = []

for seed in range(10):
    # prepare dataframes
    df_train, y_train, df_val, y_val, df_test, y_test = prepare_df(df, seed=seed)
    
    # Prepare training and validation data
    X_train = prepare_X(df_train)
    X_val = prepare_X(df_val)
    
    # Train model
    w_0, w = train_linear_regression(X_train, y_train)
    
    # Predictions
    y_pred_train = w_0 + X_train.dot(w)
    y_pred_val = w_0 + X_val.dot(w)
    
    # RMSE
    train_rmse = rmse(y_train, y_pred_train)
    val_rmse = rmse(y_val, y_pred_val)

    print(f"seed:{seed}, train RMSE: {train_rmse:.3f}, val RMSE: {val_rmse:.3f}")

    result_list.append({
        'seed': seed,
        'train_rmse': train_rmse,
        'val_rmse': val_rmse
    })


# convert to DataFrame
df_results = pd.DataFrame(result_list)
display(df_results)

# compute standard deviation of validation RMSE
std = np.std(df_results['val_rmse'])
print("Standard deviation of validation RMSE:", round(std, 3))


seed:0, train RMSE: 0.517, val RMSE: 0.521
seed:1, train RMSE: 0.515, val RMSE: 0.521
seed:2, train RMSE: 0.521, val RMSE: 0.523
seed:3, train RMSE: 0.520, val RMSE: 0.516
seed:4, train RMSE: 0.517, val RMSE: 0.511
seed:5, train RMSE: 0.520, val RMSE: 0.528
seed:6, train RMSE: 0.512, val RMSE: 0.531
seed:7, train RMSE: 0.525, val RMSE: 0.509
seed:8, train RMSE: 0.524, val RMSE: 0.515
seed:9, train RMSE: 0.522, val RMSE: 0.513


,seed,train_rmse,val_rmse
0,0,0.517,0.521
1,1,0.515,0.521
2,2,0.521,0.523
3,3,0.520,0.516
4,4,0.517,0.511
5,5,0.520,0.528
6,6,0.512,0.531
7,7,0.525,0.509
8,8,0.524,0.515
9,9,0.522,0.513


Standard deviation of validation RMSE: 0.007


### Question 6

* Split the dataset like previously, use seed 9.
* Combine train and validation datasets.
* Fill the missing values with 0 and train a model with `r=0.001`. 
* What's the RMSE on the test dataset?

Options:

- 0.15
- **`0.515`**
- 5.15
- 51.5



In [120]:

# Step 1: Split using seed 9
df_train, y_train, df_val, y_val, df_test, y_test = prepare_df(df, seed=9)

# Step 2: Combine train + validation
df_full_train = pd.concat([df_train, df_val]).reset_index(drop=True)
y_full_train = np.concatenate([y_train, y_val])

# Step 3: Prepare features (fill missing with 0)
X_full_train = prepare_X(df_full_train)
X_test = prepare_X(df_test)

# Step 4: Train model with regularization r = 0.001
w_0, w = train_linear_regression_reg(X_full_train, y_full_train, r=0.001)

# Step 5: Evaluate on test data
y_pred = w_0 + X_test.dot(w)
test_rmse = rmse(y_test, y_pred)

print("RMSE on test dataset:", round(test_rmse, 3))


RMSE on test dataset: 0.516


## Submit the results

* Submit your results here: https://courses.datatalks.club/ml-zoomcamp-2025/homework/hw02
* If your answer doesn't match options exactly, select the closest one
